In [1]:
import cv2
import numpy as np
from keras.models import model_from_json
import os

# Create the 'test' folder if it doesn't exist
output_folder = 'test'
os.makedirs(output_folder, exist_ok=True)

# Dictionary to map emotion indices to emotion names
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# Load the trained emotion detection model from JSON file
json_file = open('emotion-model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

# Load the weights into the model
emotion_model.load_weights("emotion-model.h5")
print("Loaded model from disk")

# Variables to store emotion percentages and frame count
total_frames_webcam = 0
emotion_occurrences_webcam = {emotion: 0 for emotion in emotion_dict.values()}
total_frames_ipcam = 0
emotion_occurrences_ipcam = {emotion: 0 for emotion in emotion_dict.values()}

# Set the maximum number of frames to process (adjust as needed)
max_frames = 200

# Define the video writers to save the frames as videos
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video_filename_webcam = os.path.join(output_folder, 'output_video_webcam.avi')
out_webcam = cv2.VideoWriter(output_video_filename_webcam, fourcc, 20.0, (1280, 720))

output_video_filename_ipcam = os.path.join(output_folder, 'output_video_ipcam.avi')
out_ipcam = cv2.VideoWriter(output_video_filename_ipcam, fourcc, 20.0, (1280, 720))

def detect_emotion_from_webcam(total_frames):
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()

        # Check if the frame is None
        if frame is None:
            print("Error reading frame. Exiting...")
            break

        frame = cv2.resize(frame, (1280, 720))
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        face_detector = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
        num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in num_faces:
            roi_gray_frame = gray_frame[y:y + h, x:x + w]
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

            emotion_prediction = emotion_model.predict(cropped_img)
            maxindex = int(np.argmax(emotion_prediction))
            emotion_name = emotion_dict[maxindex]

            emotion_occurrences_webcam[emotion_name] += 1
            total_frames += 1

            # Draw a green rectangle around the face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Display the predicted emotion on the frame
            cv2.putText(frame, emotion_name, (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        out_webcam.write(frame)

        cv2.imshow('Emotion Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q') or total_frames >= max_frames:
            break

    cap.release()
    cv2.destroyAllWindows()  # Close the OpenCV window when done

    # Display total emotion percentages
    emotion_percentages = {emotion: (count / total_frames) * 100 for emotion, count in emotion_occurrences_webcam.items() if count > 0}
    print("\nTotal Emotion Percentages from Webcam:")
    for emotion, percentage in emotion_percentages.items():
        print(f"{emotion}: {percentage:.2f}%")

    return total_frames  # Return the updated total_frames value

def detect_emotion_from_camera_ip(ip_address, total_frames):
    cap = cv2.VideoCapture(f"http://{ip_address}/video")

    while True:
        ret, frame = cap.read()

        # Check if the frame is None
        if frame is None:
            print("Error reading frame. Exiting...")
            break

        frame = cv2.resize(frame, (1280, 720))
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        face_detector = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
        num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in num_faces:
            roi_gray_frame = gray_frame[y:y + h, x:x + w]
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

            emotion_prediction = emotion_model.predict(cropped_img)
            maxindex = int(np.argmax(emotion_prediction))
            emotion_name = emotion_dict[maxindex]

            emotion_occurrences_ipcam[emotion_name] += 1
            total_frames += 1

            # Draw a green rectangle around the face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Display the predicted emotion on the frame
            cv2.putText(frame, emotion_name, (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        out_ipcam.write(frame)

        cv2.imshow('Emotion Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q') or total_frames >= max_frames:
            break

    cap.release()
    cv2.destroyAllWindows()  # Close the OpenCV window when done

    # Display total emotion percentages
    emotion_percentages = {emotion: (count / total_frames) * 100 for emotion, count in emotion_occurrences_ipcam.items() if count > 0}
    print("\nTotal Emotion Percentages from Camera IP:")
    for emotion, percentage in emotion_percentages.items():
        print(f"{emotion}: {percentage:.2f}%")

    return total_frames  # Return the updated total_frames value

# Main program
try:
    while True:
        print("Options:")
        print("1. Use the default webcam for emotion detection")
        print("2. Connect to a camera device using IP address for emotion detection")
        print("3. Quit")

        choice = input("Enter your choice (1/2/3): ")

        if choice == '1':
            total_frames_webcam = detect_emotion_from_webcam(total_frames_webcam)
        elif choice == '2':
            ip_address = input("Enter the camera IP address: ")
            total_frames_ipcam = detect_emotion_from_camera_ip(ip_address, total_frames_ipcam)
        elif choice == '3':
            break
        else:
            print("Invalid choice. Please enter 1, 2, or 3.")

except KeyboardInterrupt:
    print("\nProgram interrupted. Exiting...")

finally:
    # Release the video writers
    out_webcam.release()
    out_ipcam.release()

    # Release the webcam and close all windows
    cv2.destroyAllWindows()

Loaded model from disk
Options:
1. Use the default webcam for emotion detection
2. Connect to a camera device using IP address for emotion detection
3. Quit
Enter your choice (1/2/3): 2
Enter the camera IP address: 192.168.1.5:4747
1/1 [==============================] - 0s 16ms/step

Total Emotion Percentages from Camera IP:
Angry: 9.76%
Fearful: 4.07%
Happy: 18.70%
Neutral: 46.34%
Surprised: 21.14%
Options:
1. Use the default webcam for emotion detection
2. Connect to a camera device using IP address for emotion detection
3. Quit
Enter your choice (1/2/3): 1
1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 31ms/step

Total Emotion Percentages from Webcam:
Angry: 0.51%
Happy: 40.82%
Neutral: 45.41%
Sad: 3.06%
Surprised: 10.20%
Options:
1. Use the default webcam for emotion detection
2. Connect to a camera device using IP address for emotion detection
3. Quit
Enter your choice (1/2/3): 3
